# India's Prison Panel Dataset (2001-18)

A fallout of my interest in judicial politics in India has been scouring the National Crime Record Bureau (NCRB) website over the last few weeks. The website has a LOT of data, spanning from 1998 to 2018. In keeping with traditional bureaucratic rituals to torment data users though, none of the uploaded data can be manipulated. So I have taken it upon myself to spend this summer turning all this data into a clean panel dataset that can be downloaded into a .csv format. 

Of course, it'd have been ideal if the data was further disaggregated but I'm seeing this mostly as an exercise to hone my web scraping skills and not so much econometric skills. 

In [ ]:
! pip uninstall numpy -y
! pip install numpy
! pip install camelot-py[cv]

In [1579]:
## Loading packages 

import pandas as pd
import numpy as np
import requests
import re
import urllib.request, urllib.parse, urllib.error
import ssl
import tabula
import PyPDF2
import statistics

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from itertools import islice
from dateutil.parser import parse
from itertools import groupby
from tabula import read_pdf
from PyPDF2 import PdfFileReader
from io import BytesIO
from statistics import mode 

In [1581]:
## Ignore SSL cerification

ssl._create_default_https_context = ssl._create_unverified_context

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [1582]:
## Listing URLs to access NCRB website

main_url    =  "https://ncrb.gov.in/table-and-chapter-contents-of-psi-reports?"
sub_url_1   =  "field_date_value%5Bvalue%5D%5Byear%5D="
sub_url_2   =  "&field_select_psi_table_content_t_value=All&items_per_page=All"

In [1583]:
## Creating function to extract URLs that link to data from the NCRB prison website

# NOTE I: All tables that relate to demographic details of the prisoners are associated with the following strings - 
    # For years 2001 - 2015 - "demographic particulars"
    # For years 2016 - 2018 - "education", "domicile", "religion", "caste"

# NOTE II: All data is stored in pdf format so extracting links based on whether the "type" is "pdf"

def demo_url_scraper(url_list, prisoner_group):

    demo_strings      = ["demographic particulars", "education", "domicile", "religion", "caste"]
    
    for year in range(2001, 2019) :
        html = urllib.request.urlopen(main_url + sub_url_1 + str(year) + sub_url_2).read()
        soup = BeautifulSoup(html, "html.parser")  
        tags = soup("a", type = "application/pdf")
        for tag in tags:    
            if prisoner_group in tag.text.lower():
                if any(s in tag.text.lower() for s in demo_strings):    
                    url_list.append(tag.text.lower())
                    url_list.append(tag.get("href", None))                
                else: continue
            else: continue

In [1584]:
convicts_url_list   = []
undertrial_url_list = []

demo_url_scraper(convicts_url_list, "convict")
demo_url_scraper(undertrial_url_list, "undertrial")

In [1585]:
## Creating function to access URLs and read PDFs to generate pandas dataframes

# NOTE I: Will work with two fresh lists since the types of tables are different in 2001-15 and 2016-18. 

def pdf_table_reader(url_list, df_list_2001_15, df_list_2016_18):
    
    for i in range(len(url_list)):
        if "demographic particulars" in url_list[i]:
            url = url_list[i+1]
            try:
                df  = tabula.read_pdf(url, pages = "all")
                df_list_2001_15.append(df)
            except Exception as e:
                print("Error{}".format(e))         
        elif ("education" in url_list[i]) or ("domicile" in url_list[i]) or ("religion" in url_list[i]) or ("caste" in url_list[i]):
            url = url_list[i+1]
            try:
                df  = tabula.read_pdf(url, pages = "all")
                df_list_2016_18.append(df)
            except Exception as e:
                print("Error {}".format(e), url)                     

In [1586]:
convicts_df_list_2001_15 = []
convicts_df_list_2016_18 = []

undertrial_df_list_2001_15 = []
undertrial_df_list_2016_18 = []

pdf_table_reader(convicts_url_list,   convicts_df_list_2001_15,   convicts_df_list_2016_18)

pdf_table_reader(undertrial_url_list, undertrial_df_list_2001_15, undertrial_df_list_2016_18)

Got stderr: Jun 01, 2020 3:30:25 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font Arial,Bold are not implemented in PDFBox and will be ignored
Jun 01, 2020 3:30:26 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font Arial,Bold are not implemented in PDFBox and will be ignored
Jun 01, 2020 3:30:27 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font Arial,Bold are not implemented in PDFBox and will be ignored
Jun 01, 2020 3:30:28 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font Arial,Bold are not implemented in PDFBox and will be ignored
Jun 01, 2020 3:30:29 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font Arial,Bold are not implemented in PDFBox and will be ignored
Jun 01, 2020 3:30:29 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenT

Error Command '['java', '-Djava.awt.headless=true', '-Dfile.encoding=UTF8', '-jar', '/Users/chetana.sabnis/anaconda3/lib/python3.7/site-packages/tabula/tabula-1.0.3-jar-with-dependencies.jar', '--pages', 'all', '--guess', '--format', 'JSON', '/var/folders/m8/9tzpbh251ps1wdqqtpq2nrsw0000gn/T/92a601aa-cef0-43d5-bad4-02b9d82ecb3a.pdf']' returned non-zero exit status 1. https://ncrb.gov.in/sites/default/files/psi_table_and_chapter_report/TABLES%202.10C%20-%202017.pdf


Error from tabula-java:
Error: Error: End-of-File, expected line




Error Command '['java', '-Djava.awt.headless=true', '-Dfile.encoding=UTF8', '-jar', '/Users/chetana.sabnis/anaconda3/lib/python3.7/site-packages/tabula/tabula-1.0.3-jar-with-dependencies.jar', '--pages', 'all', '--guess', '--format', 'JSON', '/var/folders/m8/9tzpbh251ps1wdqqtpq2nrsw0000gn/T/6b1a6744-3bfa-4f44-870d-f296d9bd2f89.pdf']' returned non-zero exit status 1. https://ncrb.gov.in/sites/default/files/psi_table_and_chapter_report/TABLES%202.10D%20-%202017.pdf


Got stderr: Jun 01, 2020 3:33:17 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font Arial,Bold are not implemented in PDFBox and will be ignored
Jun 01, 2020 3:33:18 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font Arial are not implemented in PDFBox and will be ignored
Jun 01, 2020 3:33:18 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font Arial,Bold are not implemented in PDFBox and will be ignored
Jun 01, 2020 3:33:19 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font Arial are not implemented in PDFBox and will be ignored
Jun 01, 2020 3:33:20 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font Arial,Bold are not implemented in PDFBox and will be ignored
Jun 01, 2020 3:33:20 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout

Error Command '['java', '-Djava.awt.headless=true', '-Dfile.encoding=UTF8', '-jar', '/Users/chetana.sabnis/anaconda3/lib/python3.7/site-packages/tabula/tabula-1.0.3-jar-with-dependencies.jar', '--pages', 'all', '--guess', '--format', 'JSON', '/var/folders/m8/9tzpbh251ps1wdqqtpq2nrsw0000gn/T/1fc054fb-ecc9-4ae2-9bb2-90ef93408b5f.pdf']' returned non-zero exit status 1. https://ncrb.gov.in/sites/default/files/psi_table_and_chapter_report/TABLES%202.11C%20-%202017.pdf


Error from tabula-java:
Error: Error: End-of-File, expected line




Error Command '['java', '-Djava.awt.headless=true', '-Dfile.encoding=UTF8', '-jar', '/Users/chetana.sabnis/anaconda3/lib/python3.7/site-packages/tabula/tabula-1.0.3-jar-with-dependencies.jar', '--pages', 'all', '--guess', '--format', 'JSON', '/var/folders/m8/9tzpbh251ps1wdqqtpq2nrsw0000gn/T/db26718b-9ebd-4608-9845-f192da0d1a8c.pdf']' returned non-zero exit status 1. https://ncrb.gov.in/sites/default/files/psi_table_and_chapter_report/TABLES%202.11D%20-%202017.pdf


Got stderr: Jun 01, 2020 3:37:02 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font Arial,Bold are not implemented in PDFBox and will be ignored
Jun 01, 2020 3:37:03 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font Arial are not implemented in PDFBox and will be ignored
Jun 01, 2020 3:37:03 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font Arial,Bold are not implemented in PDFBox and will be ignored
Jun 01, 2020 3:37:04 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font Arial are not implemented in PDFBox and will be ignored
Jun 01, 2020 3:37:04 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font Arial,Bold are not implemented in PDFBox and will be ignored
Jun 01, 2020 3:37:04 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout

In [1587]:
# Creating a function to clean the dataframes that have been listed in dfs_2001_15

# NOTE I: Each object within the raw list has dataframes on education, religion/caste and domicile. 
# Will clean the list of dataframes in view of this. 

def clean_demo_tables_2001_15 (df_list, cleaned_df_list) :

    count            = 0  
    year             = 2000
    state_names      = ["Andhra Pradesh", "Arunachal Pradesh", "Assam", "Bihar", "Chhattisgarh", "Goa", "Gujarat",
                        "Haryana", "Himachal Pradesh", "Jammu and Kashmir", "Jharkhand", "Karnataka", "Kerala",
                        "Madhya Pradesh", "Maharashtra", "Manipur", "Meghalaya", "Mizoram", "Nagaland", "Odisha",
                        "Punjab", "Rajasthan", "Sikkim", "Tamil Nadu", "Tripura", "Uttar Pradesh", "Uttarakhand", 
                        "West Bengal", "A and N Islands", "Chandigarh", "D and N Haveli",
                        "Daman and Diu", "Delhi", "Lakshadweep", "Puducherry"]

    
    for dfs in df_list:
        
        # determining year for which data was collected
        year  = year + 1
        
        # tracking the dataframe within each list object to check whether it has data on edu / socio / residence
        # every first df is education
        # every second df is socio
        # every third df is residence (except for in year 2001; modifications made below)
        count = 0  
        
        for i in range(len(dfs)):
            df           = dfs[i]
           
            # dropping na variables
            df = df.dropna(axis = 0, how = "all")
            
            # tracking count
            count        = count + 1
            
            # converting all columns into string variables
            df = df.astype(str)
            
            # since many of the variables have been compressed, will need to split them.
            # given the way the data's organized though, I won't be able to split values without
            # splitting states name.
            # Splitting state names will not be consistent across all state values (e.g. Uttar Pradesh will split,
            # while Karnataka won't) and consequently, to account for split in some instances,
            # the numeric values belonging to a single variable will end up spreading across different columns.
            # In order to avoid this, will revise state names so that they cannot be split and consequently,
            # will be unable to push numeric values into adjacent columns. 
         
            halving_states = [" PRADESH", " PRADESH*", " & KASHMIR", " NADU", " BENGAL", " & N ISLANDS", 
                              " &KASHMIR", " & N HAVELI", " & DIU", "TOTAL "]   
        
            for i in range(len(halving_states)):
                df.iloc[:,1] = df.iloc[:,1].str.replace(halving_states[i], "", regex = True)
                df.iloc[:,0] = df.iloc[:,0].str.replace(halving_states[i], "", regex = True)
            
            # splitting variables that have been compressed into a single variable
            df = pd.concat([df[x].str.split(" ", expand = True) for x in df], axis = 1, keys = df)
            
            # dropping rows that have no relevant information
            irrelevant_rows = ["Caste", "SL\.", "Sl\.", "NO\.", "No\.", "Prison", "\(1\)", "nan", "\(STATES\)",
                              "\(UTS\)", "\(ALL\-INDIA\)", "\(UTs\)", "DOMICILE"]
            
            for i in range(len(irrelevant_rows)): 
                df = df[~ df.iloc[:, 0].str.contains(irrelevant_rows[i])]            

            #replacing hyphen with NA
            nan_rows = ["nan", "-", "NA"]
            
            for i in range(len(nan_rows)):
                df = df.replace(nan_rows[i], np.nan)
            
            # dropping columns which have only null values
            df = df.dropna(axis = 1, how = "all")            

            # dropping column which has serial numbers
            df = df.drop([df.columns[0]], axis = "columns")             
                
            # correcting for missing data [think tabula didn't end up scraping these parts]
            if len(df.index) == 33:
                if year == 2001:
                    df.loc[-2] = ["Andhra Pradesh", "8658", "172", "12", "8842"] 
                    df.loc[-1] = ["Arunachal Pradesh", None, None, None, None]
                    df.index   = df.index + 2           # shifting index                    
                    df.sort_index(inplace = True)       # sorting by index

                    
                elif year == 2002:
                    df.loc[-2] = ["Andhra Pradesh", "8339", "208", "2", "8549"]
                    df.loc[-1] = ["Arunachal Pradesh", None, None, None, None]                    
                    df.index   = df.index + 2           # shifting index                    
                    df.sort_index(inplace = True)       # sorting by index
                    
            if len(df.index) == 31:
                if year == 2001:
                    df.loc[-4] = ["Andhra Pradesh", "4181", "37", "3", "4221"]                    
                    df.loc[-3] = ["Arunachal Pradesh", None, None, None, None]                    
                    df.loc[-2] = ["Assam", "2366", "36", "1", "2403"]
                    df.loc[-1] = ["Bihar", "5042", "15", "8", "5065"]                    
                    df.index   = df.index + 4           # shifting index                    
                    df.sort_index(inplace = True)       # sorting by index

            if len(df.index) == 1:
                if year == 2012:
                    df.loc[0] = ["ANDHRA", "8364", "160", "27", "8551"]
                    df.loc[1] = ["ARUNACHAL PRADESH", "28", "39", "0", "67"]
                    df.loc[2] = ["ASSAM", "4330", "738", "30", "5098"]
                    df.loc[3] = ["BIHAR", "23721", "538", "130", "24389"]
                    df.loc[4] = ["CHHATTISGARH", "8308", "487", "4", "8799"]
                    df.loc[5] = ["GOA", "141", "161", "34", "336"]
                    df.loc[6] = ["GUJARAT", "5074", "1459", "80", "6613"]
                    df.loc[7] = ["HARYANA", "8558", "1658", "35", "10251"]
                    df.loc[8] = ["HIMACHAL PRADESH", "560", "186", "18", "764"]
                    df.loc[9] = ["JAMMU", "1791", "150", "66", "2007"]
                    df.loc[10] = ["JHARKHAND", "12349", "686", "0", "13035"]
                    df.loc[11] = ["KARNATAKA", "8626", "267", "47", "8940"]
                    df.loc[12] = ["KERALA", "3553", "492", "120", "4165"]
                    df.loc[13] = ["MADHYA PRADESH", "17094", "522", "3", "17619"]
                    df.loc[14] = ["MAHARASHTRA", "12725", "2863", "838", "16426"]
                    df.loc[15] = ["MANIPUR", "493", "3", "31", "527"]
                    df.loc[16] = ["MEGHALAYA", "464", "101", "40", "605"]
                    df.loc[17] = ["MIZORAM", "485", "43", "0", "528"]
                    df.loc[18] = ["NAGALAND", "200", "53", "0", "253"]
                    df.loc[19] = ["ODISHA", "9212", "0", "25", "9237"]
                    df.loc[20] = ["PUNJAB", "12954", "2388", "31", "15373"]
                    df.loc[21] = ["RAJASTHAN", "12051", "1054", "65", "13170"]
                    df.loc[22] = ["SIKKIM", "103", "39", "6", "148"]
                    df.loc[23] = ["TAMIL", "7678", "252", "64", "7994"]
                    df.loc[24] = ["TRIPURA", "261", "57", "34", "352"]
                    df.loc[25] = ["UTTAR", "52272", "1386", "163", "53821"]
                    df.loc[26] = ["UTTARAKHAND", "1310", "552", "0", "1862"]
                    df.loc[27] = ["WEST", "11723", "735", "1519", "13977"]
                    df.loc[28] = ["A", "101", "0", "265", "366"]
                    df.loc[29] = ["CHANDIGARH", "283", "186", "4", "473"]
                    df.loc[30] = ["D", "23", "14", "0", "37"]
                    df.loc[31] = ["DAMAN", "5", "6", "0", "11"]
                    df.loc[32] = ["DELHI", "6193", "2395", "299", "8887"]                                        
                    df.loc[33] = ["LAKSHADWEEP", "0", "0", "0", "0"]
                    df.loc[34] = ["PUDUCHERRY", "135", "35", "6", "176"]
                    df.sort_index(inplace = True)       # sorting by index
                    
            # renaming state names                                
            if len(df.index) == 35:
                df.iloc[:,0] = state_names
            elif len(df.index) == 36:
                state_names.insert(24, "Telanagana")     # inserting "Telangana" into state_names list 
                df.iloc[:,0] = state_names
                state_names.remove("Telanagana")         # removing "Telangana" to revert to original state_names
        
            # adding year column
            df["year"] = str(year)
            
            # cleaning df based on whether it's data on education / socio / residence
            
            # education_df
            if count == 1:
                education_df = df   

                # renaming variables
                education_df.columns = ["state", "illiterate", "below_x", "above_x_and_below_grad", 
                                        "grad", "post_grad", "tech_degree_or_diploma", "education_total", "year"]
                
            # socio_df    
            if count == 2:
                socio_df     = df

                
                # certain columns haven't split into 13 columns as a result for some rows the values are in the wrong cols
                # will be correcting this manually below.
                if len(socio_df.columns) == 14:
                    if mode(socio_df.iloc[:,13]) == "2002":
                        
                        # for convicts dfs
                        reference_values_convicts_1 = ["27", "5", "3", "2", "30", "2", "13", "0", "119"]
                        reference_values_convicts_2 = ["1", "6", "9", "6", "2", "2", "0", "2", "6"]                        

                        for i in range(len(reference_values_convicts_1)):
                            
                            socio_df.iloc[:,2] = np.where((socio_df.iloc[:,1] == reference_values_convicts_1[i]) & (socio_df.iloc[:,3] == reference_values_convicts_2[i]),
                                                           socio_df.iloc[:,2].fillna(socio_df.iloc[:,3]),
                                                           socio_df.iloc[:,2])
                                    
                            socio_df.iloc[:,3] = np.where((socio_df.iloc[:,1] == reference_values_convicts_1[i]) & (socio_df.iloc[:,3] == reference_values_convicts_2[i]),
                                                           None,
                                                           socio_df.iloc[:,3])
                            
                        # for undertrial dfs
                        reference_values_undertrial_1 = ["36", "19", "5", "86"]
                        reference_values_undertrial_2 = ["5", "0", "8", "6"]
                        
                        for i in range(len(reference_values_undertrial_1)):
                           
                                socio_df.iloc[:,2] = np.where((socio_df.iloc[:,1] == reference_values_undertrial_1[i]) & (socio_df.iloc[:,3] == reference_values_undertrial_2[i]),
                                                     socio_df.iloc[:,2].fillna(socio_df.iloc[:,3]),
                                                     socio_df.iloc[:,2])
                                    
                                socio_df.iloc[:,3] = np.where((socio_df.iloc[:,1] == reference_values_undertrial_1[i]) & (socio_df.iloc[:,3] == reference_values_undertrial_2[i]),
                                                     None,
                                                     socio_df.iloc[:,3])
                    else: 
                        socio_df.loc[:,4]  = socio_df.iloc[:,4].fillna(socio_df.iloc[:,5], inplace = True)
                        
                        col_range_1        = list(range(5,10))
                        col_range_2        = list(range(4,9))
                        col_range_3        = list(range(6,11))
                        
                        for i in range(len(col_range_1)) :
                            socio_df.iloc[:,col_range_1[i]] = np.where(socio_df.iloc[:,col_range_2[i]] == socio_df.iloc[:,col_range_1[i]],
                                                                   socio_df.iloc[:,col_range_3[i]],
                                                                   socio_df.iloc[:,col_range_1[i]])
                            
                        socio_df.iloc[:, 10] = np.where(socio_df.iloc[:,10] == socio_df.iloc[:,9],
                                                        None,
                                                        socio_df.iloc[:, 10])
                        
                    socio_df = socio_df.dropna(axis = 1, how = "all")

                #renaming columns
                socio_df.columns = ["state", "hindu", "muslim", "sikh", "christian", "religion_others", 
                                    "religion_total", "sc", "st", "obc", "caste_others", "caste_total", "year"]
                
        # domicile_df
        if (count == 3 and len(df.index) > 34) or (count == 4 and len(df.index) > 34):
            domicile_df = df
            
            #renaming columns
            domicile_df.columns = ["state", "belongs to state", "belongs to other state", 
                                    "belongs to other country", "total", "year"]
         
        else: print(count, year, df)

        merged_1 = pd.merge(left = education_df, right = socio_df, left_on = ["state", "year"], right_on = ["state", "year"])
        merged   = pd.merge(left = merged_1, right = domicile_df,  left_on = ["state", "year"], right_on = ["state", "year"])

        # appending merged dfs
        cleaned_df_list.append(merged)
     

In [1588]:
# cleaned list of datasets on convicts between 2001-15
convicts_cleaned_df_list_2001_15 = []
clean_demo_tables_2001_15(convicts_df_list_2001_15, convicts_cleaned_df_list_2001_15)

In [1589]:
# cleaned list of datasets on undertrials between 2001-15
undertrial_cleaned_df_list_2001_15 = []
clean_demo_tables_2001_15(undertrial_df_list_2001_15, undertrial_cleaned_df_list_2001_15)

In [1590]:
# Before cleaning each df, will clean the overall dfs_2016_18 lists since there were issues while scraping 

# The lists have the follow issues - 
# 1 - Contain redundant elements. For instance, NCRB has accidentally listed a table on their website 
# as "education of undertrial" when it was "education of detenue", consequently, 
# there are two dataframes that provide "education of undertrial" although there's actually only one dataframe that has the actual data.
# 2 - Don't contain some of the necessary elements (like the religious and caste data in 2017)

# Fixing Issue 1:
revised_undertrial_df_list_2016_18 = undertrial_df_list_2016_18[:4] + undertrial_df_list_2016_18[5:]

# Fixing Issue 2:

## convicts ##

# religion data on convicts in 2017
convicts_religio_2017 = pd.read_excel(r'/Users/chetana.sabnis/Documents/06_pythonScripts/03_NCRBScraping/TABLES 2.10C - 2017.xlsx')
revised_convicts_df_list_2016_18   = convicts_df_list_2016_18[:4] + [convicts_religio_2017] + convicts_df_list_2016_18[4:]

# caste data on convicts in 2017
convicts_caste_2017   = pd.read_excel(r'/Users/chetana.sabnis/Documents/06_pythonScripts/03_NCRBScraping/TABLES 2.10D - 2017.xlsx')
revised_convicts_df_list_2016_18   = revised_convicts_df_list_2016_18[:5] + [convicts_caste_2017] + revised_convicts_df_list_2016_18[5:]

## undertrial ##

# religion data on undertrial in 2017
undertrial_religio_2017 = pd.read_excel(r'/Users/chetana.sabnis/Documents/06_pythonScripts/03_NCRBScraping/TABLES 2.11C - 2017.xlsx')
revised_undertrial_df_list_2016_18 = revised_undertrial_df_list_2016_18[:4] + [undertrial_religio_2017] + revised_undertrial_df_list_2016_18[4:]

# caste data on undertrial in 2017
undertrial_caste_2017 = pd.read_excel(r'/Users/chetana.sabnis/Documents/06_pythonScripts/03_NCRBScraping/TABLES 2.11D - 2017.xlsx')
revised_undertrial_df_list_2016_18 = revised_undertrial_df_list_2016_18[:5] + [undertrial_caste_2017] + revised_undertrial_df_list_2016_18[5:]


In [1651]:
# Creating a function to clean the dataframes that have been listed in dfs_20016_18

def clean_demo_tables_2016_18 (df_list, cleaned_df_list) :
    count            = 0  
    
    state_names      = ["Andhra Pradesh", "Arunachal Pradesh", "Assam", "Bihar", "Chhattisgarh", "Goa", "Gujarat",
                        "Haryana", "Himachal Pradesh", "Jammu and Kashmir", "Jharkhand", "Karnataka", "Kerala",
                        "Madhya Pradesh", "Maharashtra", "Manipur", "Meghalaya", "Mizoram", "Nagaland", "Odisha",
                        "Punjab", "Rajasthan", "Sikkim", "Tamil Nadu", "Telangana", "Tripura", "Uttar Pradesh", 
                        "Uttarakhand", "West Bengal", "A and N Islands", "Chandigarh", "D and N Haveli",
                        "Daman and Diu", "Delhi", "Lakshadweep", "Puducherry"]

    
    for i in range(len(df_list)):
        
        # tracking count
        count        = count + 1   

        if count == 5 or count == 6:
            df = df_list[i]
        else: 
            df = df_list[i][0]

        # dropping na variables
        df = df.dropna(axis = 1, how = "all")        
                    
        # converting all columns into string variables
        df = df.astype(str)
            
        # since many of the variables have been compressed, will need to split them.
        # given the way the data's organized though, I won't be able to split values without
        # splitting states name.
        # Splitting state names will not be consistent across all state values (e.g. Uttar Pradesh will split,
        # while Karnataka won't) and consequently, to account for split in some instances,
        # the numeric values belonging to a single variable will end up spreading across different columns.
        # In order to avoid this, will revise state names so that they cannot be split and consequently,
        # will be unable to push numeric values into adjacent columns. 
         
        halving_states = [" PRADESH", " PRADESH*", " & KASHMIR", " NADU", " BENGAL", " & N ISLANDS", 
                          " &KASHMIR", " & N HAVELI", " & DIU", "TOTAL ", " BENGAL#", " #"]   
        
        for i in range(len(halving_states)):     
            df.iloc[:,1] = df.iloc[:,1].str.replace(halving_states[i], "", regex = True)
            df.iloc[:,0] = df.iloc[:,0].str.replace(halving_states[i], "", regex = True)
        
        # splitting variables that have been compressed into a single variable
        df = pd.concat([df[x].str.split(" ", expand = True) for x in df], axis = 1, keys = df)
        
            
        # dropping rows that have no relevant information
        irrelevant_rows = ["Caste", "SL\.", "Sl\.", "NO\.", "No\.", "Prison", "\(1\)", "nan", "\(STATES\)",
                            "\(UTS\)", "\(ALL\-INDIA\)", "\(UTs\)", "DOMICILE", "o\.", "", "NaN"]
            
        for i in range(len(irrelevant_rows)): 
            df = df[~ df.iloc[:, 0].str.contains(irrelevant_rows[i])]
                
        
        #replacing hyphen with NA
        nan_rows = ["nan", "-", "NA", "NaN"]
            
        for i in range(len(nan_rows)):
            df = df.replace(nan_rows[i], np.nan)            
            
            
        # dropping columns which have only null values
        df = df.dropna(axis = 1, how = "all") 
        

        # dropping column which has serial numbers
        df = df.drop([df.columns[0]], axis = "columns")       
        
        # renaming state names                                
        if len(df.index) == 36:
            df.iloc[:,0] = state_names
        else: print(df)
            
        # creating year variable
        if count == 1 or count == 2:
            year = 2016
            df["year"] = str(year)
        if count == 3 or count == 4 or count == 5 or count == 6:
            year = 2017
            df["year"] = str(year)
        if count == 7 or count == 8 or count == 9 or count == 10:
            year = 2018
            df["year"] = str(year)

        #education_df
        if count == 1 or count == 3 or count == 7:
            education_df = df
            
            # renaming columns
            education_df.columns = ["state", "illiterate", "below_x", "above_x_and_below_grad", 
                                    "grad", "post_grad", "tech_degree_or_diploma", "education_total", "year"]
        
        #domicile_df
        elif count == 2 or count == 4 or count == 8:
            domicile_df = df
            
            #renaming columns
            domicile_df.columns = ["state", "belongs to state", "belongs to other state", 
                                    "belongs to other country", "total", "year"]
        
        # socio_df
        elif count == 5 or count == 9:
            religio_df = df
            
            # renaming columns            
            religio_df.columns = ["state", "hindu", "muslim", "sikh", "christian", 
                                  "religion_others", "religion_total", "year"]
            
        elif count == 6 or count == 10:
            caste_df = df
            
            # renaming columns
            caste_df.columns = ["state", "sc", "st", "obc", "caste_others", "caste_total", "year"]          
        
        # merging dfs
        
        if count == 2 or count == 6 or count == 10:
            merged   = pd.merge(left = education_df, right = domicile_df, left_on = ["state", "year"], right_on = ["state", "year"])
            if count == 6 or count == 10:
                merged_socio = pd.merge(left = caste_df, right = religio_df, left_on = ["state", "year"], right_on = ["state", "year"]) 
                merged   = pd.merge(left = merged, right = merged_socio, left_on = ["state", "year"], right_on = ["state", "year"])
                
            # appending merged dataframes  
            cleaned_df_list.append(merged)

In [1652]:
# cleaned list of datasets on undertrials between 2016-18
undertrial_cleaned_df_list_2016_18 = []
clean_demo_tables_2016_18(revised_undertrial_df_list_2016_18, undertrial_cleaned_df_list_2016_18)

# concatenating 2001_15 and 2016_18 lists
undertrial_cleaned_df_2001_18 = pd.concat([pd.concat(undertrial_cleaned_df_list_2001_15), pd.concat(undertrial_cleaned_df_list_2016_18)], axis = 0)

In [1653]:
# cleaned list of datasets on convicts between 2016-18
convicts_cleaned_df_list_2016_18 = []
clean_demo_tables_2016_18(revised_convicts_df_list_2016_18, convicts_cleaned_df_list_2016_18)

# concatenating 2001_15 and 2016_18 lists
convicts_cleaned_df_2001_18 = pd.concat([pd.concat(convicts_cleaned_df_list_2001_15), pd.concat(convicts_cleaned_df_list_2016_18)], axis = 0)

,state,illiterate,below_x,above_x_and_below_grad,grad,post_grad,tech_degree_or_diploma,education_total,year,hindu,...,religion_total,sc,st,obc,caste_others,caste_total,belongs to state,belongs to other state,belongs to other country,total
0,Andhra Pradesh,1523,1749,532,299,72,46,4221,2001,3407,...,4221,1269,425,1576,951,4221,4181,37,3,4221
1,Arunachal Pradesh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None
2,Assam,800,1195,358,44,2,4,2403,2001,1459,...,2403,279,184,559,1381,2403,2366,36,1,2403
3,Bihar,2117,1141,1017,705,64,21,5065,2001,3535,...,5065,1488,150,2206,1221,5065,5042,15,8,5065
4,Chhattisgarh,1745,2182,591,164,41,10,4733,2001,3947,...,4733,1283,1599,1164,687,4733,4504,229,0,4733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,D and N Haveli,0,0,1,0,0,0,1,2018,1,...,1,0,0,0,1,1,0,1,0,1
32,Daman and Diu,12,2,1,0,0,0,15,2018,10,...,15,2,2,5,6,15,6,9,0,15
33,Delhi,591,1243,836,186,26,41,2923,2018,2180,...,2923,613,49,748,1513,2923,1989,837,97,2923
34,Lakshadweep,0,0,0,0,0,0,0,2018,0,...,0,0,0,0,0,0,0,0,0,0


In [1657]:
# Convert into csv sheet
convicts_cleaned_df_2001_18.to_csv(r'/Users/chetana.sabnis/Documents/07_data/01_IndianPrisonData_1997_2019/convicts_demographics_2001_18.csv', index = False)
undertrial_cleaned_df_2001_18.to_csv(r'/Users/chetana.sabnis/Documents/07_data/01_IndianPrisonData_1997_2019/undertrial_demographics_2001_18.csv', index = False)